In [10]:
import pandas as pd
from pathlib import Path

In [20]:
n_sdg = 17
sdg_strings = [''.join(('sdg',str(ii))) for ii in range(1,n_sdg+1)]
# folders = [d for d in Path('analysis_data').iterdir() if d.is_dir() and 'gams_output_files' in str(d)]
folders = [Path('analysis_data/gams_output_files - io')]

In [21]:
# load all results
all_res = {}
for f in folders:
    orient = str(f).split(' - ')[1]
    all_res[orient] = {}
    paths = [ii for ii in f.glob('sdg*')]
    sdg_strings = [os.path.basename(ii).split('_')[0] for ii in paths]
    for pp, ss in zip(paths, sdg_strings):
    # for s in sdg_strings:
        #  path = list(f.glob('_'.join((s,'*'))))[0]
         all_res[orient][ss] = pd.read_csv(pp, sep='\t', index_col=0)

In [23]:
# save slacks in excel files
for orient in all_res.keys():
    path = Path('analysis_data')/''.join(('slacks_', orient, '.xlsx'))
    writer = pd.ExcelWriter(path, engine='xlsxwriter')
    for s in all_res[orient].keys():
        n_countries = all_res[orient][s].shape[0]
        all_res[orient][s].iloc[:,1:-n_countries].to_excel(writer, sheet_name=s)
    writer.save()

In [24]:
# save lambdas in excel files
for orient in all_res.keys():
    path = Path('analysis_data')/''.join(('lambdas_', orient, '.xlsx'))
    writer = pd.ExcelWriter(path, engine='xlsxwriter')
    for s in all_res[orient].keys():
        n_countries = all_res[orient][s].shape[0]
        all_res[orient][s].iloc[:,-n_countries:].to_excel(writer, sheet_name=s)
    writer.save()

In [25]:
# save efficiencies in csv files
for orient in all_res.keys():
    path = Path('analysis_data')/''.join(('eff_', orient, '.csv'))
    eff_df = pd.concat((all_res[orient][s]['eff'] for s in sdg_strings), axis=1) # read and concat
    eff_df.columns = sdg_strings
    eff_df.to_csv(path)
